In [ ]:
#извлечение информации о музыкантах с веб сайта arzamas.academy

In [1]:
import requests
req = requests.get('https://arzamas.academy/materials/1318')
req.ok

True

In [1]:
#print(req.text)

In [256]:
import re
import pandas as pd
from bs4 import BeautifulSoup

soup = BeautifulSoup(req.text, "html.parser")
data = soup.html.body

script = data.findAll('script')
text = script[7].contents[0]
data_str = re.findall(r'\{[^\}]*\}', text) #find {}
data_str = data_str[2 : len(data_str) - 62]
#print(len(data_str))
data_dict = list()
for data in data_str:
    data = data.replace("false", "False")
    data = data.replace("true", "True")
    data = re.sub(r'\<[^>]*\>', '', data) #html remove
    data = re.sub(r'nbsp|mdash|[\n\b\r&]', '', data)
    data = re.sub(r';', ' ', data)
    data = re.sub(r'Слушайте|\—[\s\S]*Музыке', '', data)
    val = eval(data) 
    data_dict.append(val)
keys = []
for key in data_dict[0].keys():
    keys.append(key)
data_rec = []
for rec in data_dict:
    row = []
    for key in keys:
        if rec[key] is not None:
            row.append(str(rec[key]).strip())
    data_rec.append(row)
data_df = pd.DataFrame(data_rec, columns = keys)
print(data_df.head(5))
data_df.to_csv('arzamas.csv')

              name                                               info  \
0     Элвис Пресли  Вопреки распространенному мнению, Элвис Пресли...   
1        Чак Берри  Главный темнокожий рок-н-ролльщик, самостоятел...   
2  Джерри Ли Льюис  Один из самых обаятельных персонажей ранней ро...   
3     Литтл Ричард  Творчество Литтл Ричарда   неисчерпаемый источ...   
4      Бадди Холли  Большинство звезд рок-н-ролла 1950-х с годами ...   

                                      related filterGenres  \
0  Клифф Ричард, Джеки Бренстон, Тони Шеридан   рок-н-ролл   
1     Билл Хейли, Фэтс Домино, Джонни Джонсон   рок-н-ролл   
2  Джин Винсент, Карл Перкинс, The Big Bopper   рок-н-ролл   
3  Рой Браун, Скримин Джей Хокинс, Айк Тернер   рок-н-ролл   
4      The Crickets, Рой Орбисон, Эдди Кокран   рок-н-ролл   

           relatedGenres explicit           yandex youtube decade  \
0  рок-н-ролл, рокабилли    False    Elvis Presley   False   1950   
1             рок-н-ролл    False      Chuck Berry

In [76]:
# Извлечение информции с сайта яндекс музыки https://music.yandex.ru/genre/джаз/artists
# Загрузка жанров
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

all_genre = ''
genre = 'джаз'
req = requests.get('https://music.yandex.ru/genre/' + genre + '/artists')
print(req.ok)

soup = BeautifulSoup(req.text, "html.parser")
data = soup.html.body

resultset = data.find_all('script')
result = ''
for tag in resultset:
    if tag.contents != []:
        print(len(tag.contents[0]))
        if len(tag.contents[0]) > 10000: #берем скрипт побольше
            result = tag.contents[0]
            break
#выделяем только то что в фигурных скобках
beg = result.find('{')
end = result.rfind('}')
result = result[beg:end + 1]
#print(result)

j_data = json.loads(result)

#найдем список жанров
j_norm = pd.json_normalize(j_data['genres']['structure'], meta = ['id', 'subGenres']) 
j_norm = j_norm[['id', 'subGenres']]
rows = []
for row in j_norm['subGenres']:
    try:
        rows.append([d['id'] for d in row])
    except Exception:
        rows.append([])
        
j_norm['subGenres'] = rows

nums = []
for j in j_norm.iterrows():
    num = j[0]
    j= j[1]
    if j['id'] not in j['subGenres']:
        nums.append(num)
for num in nums:
    j_norm['subGenres'][num].append(j_norm['id'][num])

print(j_norm)
j_norm.to_csv('genres.csv', index = None)

'''
#найдем артистов на странице
j_norm = pd.json_normalize(j_data['pageData']['tagData']['artists'], meta =['id']) 
j_norm = j_norm[['artist.id', 'artist.name', 'artist.genres']]
j_norm.columns = ['id', 'name', 'genres']
j_norm['current_genre'] = pd.Series([genre for g in range(j_norm.shape[0])])
#j_norm = j_norm[['current_genre', 'id', 'name', 'genres']]
print(j_norm)
'''


True
1431099
                   id                                          subGenres
0                 all                                              [all]
1                 pop  [ruspop, disco, kpop, turkishpop, uzbekpop, ja...
2             allrock  [rusrock, rock-n-roll, prog-rock, post-rock, n...
3               indie                               [local-indie, indie]
4               metal  [classicmetal, progmetal, numetal, epicmetal, ...
5         alternative  [posthardcore, hardcore, turkishalternative, a...
6          electronic                [dubstep, experimental, electronic]
7               dance        [house, techno, trance, drum-n-bass, dance]
8             hip-hop  [rusrap, foreignrap, turkishrap, israelirap, h...
9               r-n-b                                [soul, funk, r-n-b]
10               jazz                     [trad-jass, modern-jazz, jazz]
11              blues                                            [blues]
12             reggae                 

"\n#найдем артистов на странице\nj_norm = pd.json_normalize(j_data['pageData']['tagData']['artists'], meta =['id']) \nj_norm = j_norm[['artist.id', 'artist.name', 'artist.genres']]\nj_norm.columns = ['id', 'name', 'genres']\nj_norm['current_genre'] = pd.Series([genre for g in range(j_norm.shape[0])])\n#j_norm = j_norm[['current_genre', 'id', 'name', 'genres']]\nprint(j_norm)\n"

In [77]:
# загрузка списка артистов по каждому направлению
# Извлечение информции с сайта яндекс музыки https://music.yandex.ru/genre/джаз/artists
# Загрузка жанров
import re
import pandas as pd
from bs4 import BeautifulSoup
import requests
import json

all_genres = pd.read_csv('genres.csv', header = 0)
result_df = pd.DataFrame(columns = ['genre', 'subgenre', 'id', 'name', 'all_genres'])
pages = ['', '?page=1', '?page=2', '?page=3', '?page=4', '?page=5', '?page=6', '?page=7', '?page=8', '?page=9']
for row in all_genres.iterrows():
    row = row[1]
    genre = row['id']
    subgenres = eval(row['subGenres'])
    print(subgenres)
    for subgenre in subgenres:
        print(subgenre)
        for page in pages:
            req = requests.get('https://music.yandex.ru/genre/' + subgenre.strip() + '/artists' + page)
            print(req.ok)
            if req.ok:

                soup = BeautifulSoup(req.text, "html.parser")
                data = soup.html.body

                resultset = data.find_all('script')
                result = ''
                for tag in resultset:
                    if tag.contents != []:
                        #print(len(tag.contents[0]))
                        if len(tag.contents[0]) > 10000: #берем скрипт побольше
                            result = tag.contents[0]
                            break
                #выделяем только то что в фигурных скобках
                beg = result.find('{')
                end = result.rfind('}')
                result = result[beg:end + 1]
                #print(result)

                j_data = json.loads(result)

                #найдем артистов на странице
                j_norm = pd.json_normalize(j_data['pageData']['tagData']['artists'], meta =['id']) 
                j_norm = j_norm[['artist.id', 'artist.name', 'artist.genres']]
                j_norm.columns = ['id', 'name', 'all_genres']
                j_norm['subgenre'] = pd.Series([subgenre for g in range(j_norm.shape[0])])
                j_norm['genre'] = pd.Series([genre for g in range(j_norm.shape[0])])
                print(type(result))
                result_df = pd.concat([result_df, j_norm], ignore_index = True)
                print(result_df)
                print(result_df.shape)

result_df.to_csv('artist.csv', index = None)



['all']
all
True
<class 'str'>
   genre subgenre       id          name                   all_genres
0    all      all  5007577        Zivert                     [ruspop]
1    all      all    41052  Король и Шут                       [punk]
2    all      all    41191         Баста                     [rusrap]
3    all      all   666984  Artik & Asti                     [ruspop]
4    all      all  4944372       NILETTO                     [rusrap]
..   ...      ...      ...           ...                          ...
95   all      all    36784    Nickelback          [rock, alternative]
96   all      all   264454          Zell              [dance, rusrap]
97   all      all  5926594   Radio Tapok  [alternative, pop, rusrock]
98   all      all  2618492          ГРОТ                     [rusrap]
99   all      all  4027035    Cream Soda       [dance, house, rusrap]

[100 rows x 5 columns]
(100, 5)
True
<class 'str'>
    genre subgenre       id            name                 all_genres
0     

True
<class 'str'>
    genre subgenre       id               name  \
0     all      all  5007577             Zivert   
1     all      all    41052       Король и Шут   
2     all      all    41191              Баста   
3     all      all   666984       Artik & Asti   
4     all      all  4944372            NILETTO   
..    ...      ...      ...                ...   
995   all      all  3971885     V $ X V PRiNCE   
996   all      all   453480  Борис Гребенщиков   
997   all      all  4981931     Where It's ATT   
998   all      all  2891625      Северный флот   
999   all      all  5201940     Brennan Savage   

                          all_genres  
0                           [ruspop]  
1                             [punk]  
2                           [rusrap]  
3                           [ruspop]  
4                           [rusrap]  
..                               ...  
995                         [rusrap]  
996                        [rusrock]  
997      [dance, electronics,

True
<class 'str'>
     genre subgenre       id          name               all_genres
0      all      all  5007577        Zivert                 [ruspop]
1      all      all    41052  Король и Шут                   [punk]
2      all      all    41191         Баста                 [rusrap]
3      all      all   666984  Artik & Asti                 [ruspop]
4      all      all  4944372       NILETTO                 [rusrap]
...    ...      ...      ...           ...                      ...
1895   pop   ruspop  5660205  Женя Ефимова            [ruspop, pop]
1896   pop   ruspop  7051042         LAUTA       [ruspop, rap, pop]
1897   pop   ruspop  8291759         Libra                 [ruspop]
1898   pop   ruspop  7125004       UGLYBOY  [rusrap, dance, ruspop]
1899   pop   ruspop  9066614          Madi            [ruspop, pop]

[1900 rows x 5 columns]
(1900, 5)
True
<class 'str'>
     genre subgenre       id            name              all_genres
0      all      all  5007577          Zive

True
<class 'str'>
     genre    subgenre       id             name  \
0      all         all  5007577           Zivert   
1      all         all    41052     Король и Шут   
2      all         all    41191            Баста   
3      all         all   666984     Artik & Asti   
4      all         all  4944372          NILETTO   
...    ...         ...      ...              ...   
2699   pop  turkishpop  4053463         Ersan Er   
2700   pop  turkishpop  1480008  Aysel Yakupoğlu   
2701   pop  turkishpop  1020626             Ayda   
2702   pop  turkishpop   296975            Gökçe   
2703   pop  turkishpop   613339            Hepsi   

                              all_genres  
0                               [ruspop]  
1                                 [punk]  
2                               [rusrap]  
3                               [ruspop]  
4                               [rusrap]  
...                                  ...  
2699             [pop, folk, turkishpop]  
2700  [arabe

True
<class 'str'>
     genre subgenre       id              name            all_genres
0      all      all  5007577            Zivert              [ruspop]
1      all      all    41052      Король и Шут                [punk]
2      all      all    41191             Баста              [rusrap]
3      all      all   666984      Artik & Asti              [ruspop]
4      all      all  4944372           NILETTO              [rusrap]
...    ...      ...      ...               ...                   ...
3329   pop      pop    12438      Joan Osborne     [rock, pop, folk]
3330   pop      pop   878463        Little Mix                 [pop]
3331   pop      pop  3224292  Leslie Odom, Jr.  [pop, jazz, holiday]
3332   pop      pop  3681785              Lauv    [pop, dance, kpop]
3333   pop      pop   424985  Алексей Воробьёв              [ruspop]

[3334 rows x 5 columns]
(3334, 5)
True
<class 'str'>
     genre subgenre       id           name                      all_genres
0      all      all  50

...

True
<class 'str'>
                genre        subgenre       id               name  \
0                 all             all  5007577             Zivert   
1                 all             all    41052       Король и Шут   
2                 all             all    41191              Баста   
3                 all             all   666984       Artik & Asti   
4                 all             all  4944372            NILETTO   
...               ...             ...      ...                ...   
50237  classicalmusic  classicalmusic   158726       Deyan Pavlov   
50238  classicalmusic  classicalmusic  1254704       Guy Janssens   
50239  classicalmusic  classicalmusic   668193              Emika   
50240  classicalmusic  classicalmusic  1254703  Laudantes Consort   
50241  classicalmusic  classicalmusic   700630   Kiril Kondrashin   

                               all_genres  
0                                [ruspop]  
1                                  [punk]  
2                   

True
<class 'str'>
            genre subgenre       id             name  \
0             all      all  5007577           Zivert   
1             all      all    41052     Король и Шут   
2             all      all    41191            Баста   
3             all      all   666984     Artik & Asti   
4             all      all  4944372          NILETTO   
...           ...      ...      ...              ...   
50588  audiobooks  fiction  2296841        Лев Дуров   
50589  audiobooks  fiction  3682015    Андрей Мягков   
50590  audiobooks  fiction  2557956  Виктор Сергачёв   
50591  audiobooks  fiction  8524472   Сергей Шакуров   
50592  audiobooks  fiction  3460822   Вера Васильева   

                            all_genres  
0                             [ruspop]  
1                               [punk]  
2                             [rusrap]  
3                             [ruspop]  
4                             [rusrap]  
...                                ...  
50588                

True
<class 'str'>
       genre subgenre       id                                  name  \
0        all      all  5007577                                Zivert   
1        all      all    41052                          Король и Шут   
2        all      all    41191                                 Баста   
3        all      all   666984                          Artik & Asti   
4        all      all  4944372                               NILETTO   
...      ...      ...      ...                                   ...   
51138  other  holiday   455666                         Stevan Pasero   
51139  other  holiday   296950  Guy Lombardo and His Royal Canadians   
51140  other  holiday  1983430                     Paul & Marie Rein   
51141  other  holiday  3265694                             Lil' Paul   
51142  other  holiday  1128032                     Robert de Cormier   

                           all_genres  
0                            [ruspop]  
1                              [punk

True
<class 'str'>
       genre subgenre       id                  name                all_genres
0        all      all  5007577                Zivert                  [ruspop]
1        all      all    41052          Король и Шут                    [punk]
2        all      all    41191                 Баста                  [rusrap]
3        all      all   666984          Artik & Asti                  [ruspop]
4        all      all  4944372               NILETTO                  [rusrap]
...      ...      ...      ...                   ...                       ...
51897  other    other    85388             Bobby Vee      [rock, pop, holiday]
51898  other    other  1132500  The United Orchestra                 [holiday]
51899  other    other     8285            Rick Braun  [jazz, holiday, conjazz]
51900  other    other     3891        Leroy Anderson     [holiday, pop, relax]
51901  other    other    25305      Dickie Valentine      [pop, jazz, holiday]

[51902 rows x 5 columns]
(51902,

True
<class 'str'>
       genre subgenre       id           name                    all_genres
0        all      all  5007577         Zivert                      [ruspop]
1        all      all    41052   Король и Шут                        [punk]
2        all      all    41191          Баста                      [rusrap]
3        all      all   666984   Artik & Asti                      [ruspop]
4        all      all  4944372        NILETTO                      [rusrap]
...      ...      ...      ...            ...                           ...
52597  other    other  1024282    Matt Treder                     [holiday]
52598  other    other  4703583  Jess Thristan       [country, pop, holiday]
52599  other    other  1417029     Daniel May  [newage, holiday, classical]
52600  other    other  6864870     Matt Large  [foreignrap, holiday, relax]
52601  other    other    12013         Odetta         [blues, pop, holiday]

[52602 rows x 5 columns]
(52602, 5)
True
<class 'str'>
       genre 

In [1]:
#оставляем только нужные жанры
import pandas as pd
import re
artists = pd.read_csv('artist.csv', header = 0)
#выделяем латиноамериканскую в отдельный класс
nums = []
for artist in artists.iterrows():
    num = artist[0]
    artist = artist[1]
    if artist['subgenre'] == 'latin-american' and artist['genre'] == 'world':
        #print("replace latin")
        nums.append(num)
for num in nums:
    artists['genre'][num] = 'latin-american'
    
#считаем сколько исполнителей в каждом классе
genres = ['world', 'country', 'latin-american', 'blues', 'r-n-b', 'jazz', 'shanson', \
          'electronic', 'allrock', 'hip-hop', 'reggae', 'pop'] #12 жанров
summ = dict()
for genre in genres:
    summ[genre] = artists['genre'].eq(genre).sum()
    print(genre, summ[genre])

#переформатировать датафрейм
artists = artists.sample(frac = 1).reset_index(drop = True) #shuffle
print(artists)
artist_table = pd.DataFrame()
for genre in genres:
    curr_genr_artist = []
    for artist in artists.iterrows():
        num = artist[0]
        artist = artist[1]
        if artist['genre'] == genre:
            if len(curr_genr_artist) == summ[genre]:
                break
            if artist['name'] not in curr_genr_artist:
                temp = re.sub(r'\<|\>|\:|\"|\\|\/|\||\?|\*', '', artist['name']).strip()
                curr_genr_artist.append(temp)
    series = pd.Series(curr_genr_artist)
    
    artist_table[genre] = series
print(artist_table)
artist_table.to_csv('artist_table.csv', index = None)


C:\Anaconda\envs\python_37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


world 3297
country 1000
latin-american 1000
blues 679
r-n-b 2269
jazz 1817
shanson 438
electronic 2057
allrock 4475
hip-hop 3112
reggae 1589
pop 2934
            genre     subgenre       id                     name  \
0             pop       ruspop  9708948           VITA CHIKOVANI   
1            easy       lounge  2205445                 Kognitif   
2            easy       lounge  7940087                     Touf   
3             pop   turkishpop  9153518                Ali Biçim   
4      electronic      dubstep  6206971              Dylan Locke   
...           ...          ...      ...                      ...   
52797       other      holiday  1008286             Colton Dixon   
52798     country      country  4116245              Margo Price   
52799        jazz    trad-jass   283724  Ambrose & His Orchestra   
52800      reggae       reggae  3140063             Common Kings   
52801       dance  drum-n-bass  5886299              Sister Mary   

                                 

In [ ]:
#скачивание музыки 300 * 30 * 12 = 108000 песен
#431kb*108000 = 45 Gb
from yandex_music import Client
import pandas as pd
import numpy as np
import random
import os
import soundfile as sf
import librosa.util
from pydub import AudioSegment
#import audiosegment
import zipfile
import shutil
from shutil import make_archive

#from google.colab import drive
#drive.mount('/gdrive')

num_of_tracks = 30
num_of_artists = 300
pages = 3

def download_audio(filepath):
    bitrate = 192
    codec = 'mp3'
    #Известные значения codec: mp3, aac.
    #Известные значения bitrate_in_kbps: 64, 128, 192, 320.
    if(os.path.exists(filepath)): #удалить файл если существовал
        try:
            os.remove(filepath)
        except OSError as e:
            print("Ошибка: %s : %s" % (filepath, e.strerror))
            return False
    try:
        result.download(filename = filepath, codec = codec, bitrate_in_kbps = bitrate)
    except:
        print(f'Не удалось скачать файл {filepath}')
        return False
    
    return True

def convert_audio(filepath): #convert to wav and delete aac
    #print (AudioSegment.converter)
    sr = 22050
    try:
        sound = AudioSegment.from_mp3(filepath)
        '''print(f'Аудиофайл {filepath} \n громкость: {sound.dBFS}\n каналов: {sound.channels} \
        \n байтов в каждой выборке: {sound.sample_width}\n частота дискретизации: {sound.frame_rate} \
        \n байтов на кадр: {sound.frame_width}\n постоянное смещение: {sound.get_dc_offset()}')'''

        sound = sound.set_channels(1)
        sound = sound.set_sample_width(1)
        sound = sound.set_frame_rate(sr)
        #sound = sound.remove_dc_offset()

        '''print('Новые значения:')
        print(f'Аудиофайл {filepath} \n громкость: {sound.dBFS}\n каналов: {sound.channels} \
        \n байтов в каждой выборке: {sound.sample_width}\n частота дискретизации: {sound.frame_rate} \
        \n байтов на кадр: {sound.frame_width}\n постоянное смещение: {sound.get_dc_offset()}')
        #samples = sound.get_array_of_samples()'''

        #save wav
        filepath_wav = filepath[:len(filepath) - 3] + 'wav'
        sound.export(filepath_wav, format="wav")
        #delete mp3
        os.remove(filepath)
    except:
        print(f'Не удалось конвертировать файл {filepath}')
        return None
    print('Completed!')
    return filepath_wav

def trim(filepath_wav): #trim audio 10 sec
    top_db = 50 #порог тишины
    
    with sf.SoundFile(filepath_wav, 'r', closefd = True) as f:
        x = f.read()
        sr = f.samplerate
        channels = f.channels
        f.close()
    
    #убираем тишину
    try:
        x, index = librosa.effects.trim(x, top_db = top_db)
        ts = librosa.effects.split(x, top_db = top_db)
        print(ts)
        union = []
        for l in ts:
            start = l[0]
            end = l[1]
            union.append(x[start : end])
        x = np.array(union).ravel()

        step = 1  #количество секунд по которым детектируем максимум
        t = 10       #секунды длительность после обрезки
        track_time = x.shape[0] // sr #продолжительность трека
        if track_time < t:
            return False
        count = track_time // t #число шагов
        lim = t // 2

        i_max = random.randint(lim, track_time // step - lim)
        print(f' Трек {filepath_wav}\n обрезка начиная с времени {(lim + i_max) * step}\n длительность {track_time}')
        min_index = i_max - lim
        max_index = min_index + t // step

        x = x[step * sr * min_index : step * sr * max_index]
        x_norm = librosa.util.normalize(x)
    except:
        print(f'Не удалось обрезать файл {filepath_wav}')
        os.remove(filepath_wav)
        return False
    
    '''with SoundFile(file, 'r', samplerate, channels, subtype, endian, format, closefd) as f:'''
    with sf.SoundFile(filepath_wav, 'w', samplerate = sr, channels = channels) as f:
        f.write(x_norm)
        f.close()
    return True

root = os.getcwd()

artist_table = pd.read_csv('artist_table.csv', header = 0)
genres = artist_table.columns #12 жанров

f = open('pass.txt', 'r')
login, password = f.read().split(', ')
f.close()
client = Client.from_credentials(login, password)

for genre in genres:
    series = artist_table[genre]
    num_a = 0
    dirpath_genre = os.path.join(root, 'yandex_dataset', genre)
    for artist in series: 
        if num_a == num_of_artists or artist == '':
            break
        search = [client.search(artist, type_ = 'track', page = i) for i in range(pages)]
        if search[0].tracks.total < num_of_tracks:
            continue
            
        num_a = num_a + 1
        
        dirpath = os.path.join(root, 'yandex_dataset', genre, artist)
        if not(os.path.exists(dirpath)): #создать директорию, если не было
            try:
                os.makedirs(dirpath)
            except OSError:
                print ("Создать директорию %s не удалось" % dirpath)
                break
            else:
                print ("Успешно создана директория %s " % dirpath)
        
        num_t = 0
        for page in range(pages):
            for result in search[page].tracks.results:
                if num_t == num_of_tracks:
                    break 
                filename = str(num_t + 1) + '_' + artist + '_' + str(result.id) + '.mp3' #+ result.title
                filepath = os.path.join(dirpath, filename)
                if(download_audio(filepath)):
                    num_t = num_t + 1
                    filepath_wav = convert_audio(filepath)
                    if filepath_wav is not None:
                        flag = trim(filepath_wav)
                        if not flag:
                            try:
                                os.remove(filepath_wav)
                            except:
                                print('Не удалось удалить файл')
                            num_t = num_t - 1
                            continue
                    else: #неудача при конвертации
                        try:
                            os.remove(filepath)
                        except:
                            print('Не удалось удалить файл')
                        num_t = num_t - 1
                        continue
            if num_t == num_of_tracks:
                break
            print(f'Для {filename} найдено всего {num_t} треков, что меньше {num_of_tracks}')
        
        #упаковка в zip
        try:
            zipfilename = artist + '.zip'
            with zipfile.ZipFile(os.path.join(dirpath_genre, zipfilename), "w", zipfile.ZIP_DEFLATED) as newzip:
                for new_root, dirs, files in os.walk(dirpath):
                    for name in files:
                        newzip.write(os.path.join(new_root, name), name)    
                newzip.close()
            shutil.rmtree(dirpath) #удаляем папку с муз. файлами
        except:
            print ("Создать архив %s не удалось" % dirpath)

Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.account.status.Status'>; kwargs: {'advertisement': 'Оформите постоянную подписку – первый месяц бесплатно!', 'bar_below': {'alert_id': 'xxx', 'text': 'тест Подписка скоро закончится. А вместе с ней закончатся музыка и подкасты без ограничений. Не надо так.', 'bg_color': '#FF5038', 'text_color': '#FFFFFF', 'alert_type': 'Churn_SubscriptionEnd_Music', 'button': {'text': 'Оформить подписку', 'bg_color': '#FFFFFF', 'text_color': '#191919', 'uri': 'https://plus.yandex.ru/?source=music_web_churn_subscriptionend'}, 'close_button': False}}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.account.status.Status'>; kwargs: {'advertisement': 'Оформите постоянную подписку – первый месяц бесплатно!', 'bar_below': {'alert_id': 'xxx', 'text': 'Ста

Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '16f99ef9.a.116859'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '035b8c68.a.1084140'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '8cb99fe1.a.307844'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'acae6cf8.a.208487'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), tha

Успешно создана директория C:\genres_dataset\yandex_dataset\world\Rahul Sharma 
Completed!
[[      0 7721472]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\1_Rahul Sharma_2860021.wav
 обрезка начиная с времени 255
 длительность 350
Completed!
[[      0 6951936]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\2_Rahul Sharma_30354900.wav
 обрезка начиная с времени 87
 длительность 315
Completed!
[[      0 5155840]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\3_Rahul Sharma_5849320.wav
 обрезка начиная с времени 205
 длительность 233
Completed!
[[      0 8877056]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\4_Rahul Sharma_1429508.wav
 обрезка начиная с времени 201
 длительность 402
Completed!
[[      0 8906208]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\5_Rahul Sharma_10065642.wav
 обрезка начиная с времени 157
 длительность 403
Completed!
[[       0 10086912]]
 Трек C:\genres_dataset\yandex_dataset\world\Rahul Sharma\6_R

Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'd2a8f18d.a.3302055'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '1697fd7b.a.4835411'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'feafb072.a.3302058'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'd08ef4e5.a.3327748'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), 

Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'd2a8f18d.a.3302055'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '61296982.a.3302057'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'e5c4ab32.a.3302080'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '057a68c1.a.3077305'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.search.search.Search'>; kwargs: {'type_': 'track', 'page': 1, 'per_page': 20}
Found unkno

...

Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'e043bce8.a.7838478'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'f12acde9.a.5821343'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '627ff597.a.8135129'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '12656fa5.a.8134905'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.search.search.Search'>; kwargs: {'type_': 'track', 'page': 1, 'per_page': 20}
Found unkno

Успешно создана директория C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева 
Completed!
[[      0 5591256]]
 Трек C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева\1_Зайнаб Махаева_43394484.wav
 обрезка начиная с времени 203
 длительность 253
Completed!
[[      0 6387386]]
 Трек C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева\2_Зайнаб Махаева_55563712.wav
 обрезка начиная с времени 128
 длительность 289
Completed!
[[      0 5477357]]
 Трек C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева\3_Зайнаб Махаева_62838506.wav
 обрезка начиная с времени 222
 длительность 248
Completed!
[[      0 4719104]]
 Трек C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева\4_Зайнаб Махаева_55563718.wav
 обрезка начиная с времени 63
 длительность 214
Completed!
[[      0 6257664]]
 Трек C:\genres_dataset\yandex_dataset\world\Зайнаб Махаева\5_Зайнаб Махаева_41483276.wav
 обрезка начиная с времени 14
 длительность 283
Completed!
[[      0 4741120]]
 Трек C:\genres_dataset\yandex_dataset\

Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '9ddd422c.a.8599645'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'd30f3121.a.9281308'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '10ed1184.a.11764263'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.search.search.Search'>; kwargs: {'type_': 'track', 'page': 0, 'per_page': 20}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (git

Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '54ca82bf.a.10137476'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': 'fd902f9f.a.8376705'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '906e9c75.a.11370072'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue), thank you!
Type: <class 'yandex_music.album.album.Album'>; kwargs: {'storage_dir': '3c5cb198.a.9814543'}
Found unknown fields received from API! Please copy warn message and send to https://bit.ly/3dsFxyH (github issue)

Успешно создана директория C:\genres_dataset\yandex_dataset\world\Hovo 
Completed!
[[      0 5387264]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\1_Hovo_30269323.wav
 обрезка начиная с времени 73
 длительность 244
Completed!
[[       0 11561024]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\2_Hovo_35763072.wav
 обрезка начиная с времени 43
 длительность 524
Completed!
[[      0 4598976]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\3_Hovo_56518428.wav
 обрезка начиная с времени 119
 длительность 208
Completed!
[[      0 7921216]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\4_Hovo_41258219.wav
 обрезка начиная с времени 50
 длительность 359
Completed!
[[      0 7756096]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\5_Hovo_41258501.wav
 обрезка начиная с времени 30
 длительность 351
Completed!
[[      0 4919296]]
 Трек C:\genres_dataset\yandex_dataset\world\Hovo\6_Hovo_12542695.wav
 обрезка начиная с времени 106
 длительность 223
Completed!
[[      0 6065280]]